In [1]:
from fastmcp import Client
import asyncio
import nest_asyncio

# Allow nested event loops
nest_asyncio.apply()

In [2]:
async def run_llm_demo(server_path: str = "mcp_server.py"):
    async with Client(server_path) as client:
        tools = await client.list_tools()
        print(f"\ntools:\n {tools}")

        resources = await client.list_resources()
        print(f"\nresources:\n {resources}")

In [3]:
# Now asyncio.run() will work in Jupyter
asyncio.run(run_llm_demo())


tools:
 [Tool(name='calculator', description='\nPerforms arithmetic operations on two numbers.\nSupports: add, subtract, multiply, divide, power\n', inputSchema={'properties': {'operation': {'title': 'Operation', 'type': 'string'}, 'num1': {'title': 'Num1', 'type': 'number'}, 'num2': {'title': 'Num2', 'type': 'number'}}, 'required': ['operation', 'num1', 'num2'], 'type': 'object'}, annotations=None), Tool(name='trig', description='\nPerforms trigonometric operation on an angle in degree.\nSupports: sine, cosine, tangent\n', inputSchema={'properties': {'operation': {'title': 'Operation', 'type': 'string'}, 'theta': {'title': 'Theta', 'type': 'number'}}, 'required': ['operation', 'theta'], 'type': 'object'}, annotations=None), Tool(name='stock_quote', description='\nRetrieves live stock data for a given ticker symbol.\nExample: AAPL, GOOGL, MSFT, TSLA\n', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}}, 'required': ['ticker'], 'type': 'object'}, annotations=

In [4]:
async def debug_resources():
    async with Client("mcp_server.py") as client:
        print("=== Tools ===")
        tools = await client.list_tools()
        for tool in tools:
            print(f"Tool: {tool.name} - {tool.description}")
        
        print("\n=== Resources ===")
        resources = await client.list_resources()
        for resource in resources:
            print(f"Resource: {resource.uri} - {resource.description}")
            
        print("\n=== Test Reading Specific Stock Resource ===")
        try:
            # Test reading a specific ticker resource
            result = await client.read_resource("stock://AAPL")
            print("✅ Successfully read stock://AAPL")
            if hasattr(result, 'content') and result.content:
                print(f"Content: {result.content[0].text}")
            elif isinstance(result, list) and result:
                print(f"Content: {result[0].text if hasattr(result[0], 'text') else result[0]}")
            else:
                print(f"Raw result: {result}")
        except Exception as e:
            print(f"❌ Error reading stock://AAPL: {e}")
            
        print("\n=== Test Reading Server Info Resource ===")
        try:
            result = await client.read_resource("info://server")
            print("✅ Successfully read info://server")
            if hasattr(result, 'content') and result.content:
                print(f"Content: {result.content[0].text}")
            elif isinstance(result, list) and result:
                print(f"Content: {result[0].text if hasattr(result[0], 'text') else result[0]}")
            else:
                print(f"Raw result: {result}")
        except Exception as e:
            print(f"❌ Error reading info://server: {e}")

In [5]:
await debug_resources()

=== Tools ===
Tool: calculator - 
Performs arithmetic operations on two numbers.
Supports: add, subtract, multiply, divide, power

Tool: trig - 
Performs trigonometric operation on an angle in degree.
Supports: sine, cosine, tangent

Tool: stock_quote - 
Retrieves live stock data for a given ticker symbol.
Example: AAPL, GOOGL, MSFT, TSLA

Tool: health - 
Simple health check to verify server is running.

Tool: echo - 
Echo back the provided message. Useful for testing.


=== Resources ===
Resource: info://server - None

=== Test Reading Specific Stock Resource ===
✅ Successfully read stock://AAPL
Content: Apple Inc. (AAPL) - A publicly traded company

=== Test Reading Server Info Resource ===
✅ Successfully read info://server
Content: This is a demo MCP server built with FastMCP. It provides calculator and stock quote tools.
